# Generation of exogenous variables scenarios

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
import gower  
from sklearn.model_selection import ParameterGrid
from scipy.spatial.distance import cdist
import pickle
from collections import Counter

<br>

In [2]:
data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

data.head(5)

,time,temperature,relative_humidity,apparent_temperature,weather_code,region,consumption
0,2013-01-01 00:00:00,4.83,85.33,1.03,2,Auvergne-Rhône-Alpes,8173.0
1,2013-01-01 00:30:00,4.88,85.00,1.07,2,Auvergne-Rhône-Alpes,8173.0
2,2013-01-01 01:00:00,4.93,84.67,1.10,2,Auvergne-Rhône-Alpes,7944.0
3,2013-01-01 01:30:00,4.93,84.00,1.08,2,Auvergne-Rhône-Alpes,7896.0
4,2013-01-01 02:00:00,4.93,83.33,1.07,2,Auvergne-Rhône-Alpes,7882.0


<br>

In [3]:
train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

<br>

## Using weather_code


In [4]:
def generate_scenarios_code(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)

    # Filter training data to include all dates before the provided date
    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature', 'weather_code']
    
    # Standardize the data before calculating distances
    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]
    
        # Calculate Gower distances for each observation in test data compared to each observation in train subset
        distances = gower.gower_matrix(scaled_test, scaled_train_subset)

        # Select only the distances corresponding to element i of test with element i of scaled_train_subset
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_test))]

        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)

    # Find the most similar previous days in the training dataset
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]

    n_scenarios = n*(2*k+1)
    scenarios = []
    similar_days = []

    # Generate variations of plus/minus k periods of the following days to those similar days
    for idx in real_indices:
        day = train_data.iloc[idx:idx+48] # Take data from the next day with variations of ±k half hours
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                similar_days.append(day)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios, similar_days

<br>

In [5]:
# Example for Auvergne-Rhône-Alpes (2019-01-01)
target_date = datetime.date(2019, 1, 1)
scenarios, similar_days = generate_scenarios_code(train_data, 'Auvergne-Rhône-Alpes', target_date)

print('Number of scenarios:', len(scenarios))

print("\n---------------------------------------------------------- Real data ------------------------------------------------------------------------")
Auvergne_data = train_data[train_data['region'] == 'Auvergne-Rhône-Alpes']
print('\nToday:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date].head(4).to_string(index=False))
print('\nTomorrow:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date + datetime.timedelta(days=1)].head(4).to_string(index=False))

for i, escenario in enumerate(scenarios[:3]):
    print(f"\n---------------------------------------------------------- Scenario {i+1} ------------------------------------------------------------------------")
    print('\nSimilar day:')
    print(similar_days[i].head(4).to_string(index=False)) 
    print('\nNext day:')
    print(escenario.head(4).to_string(index=False)) 

Number of scenarios: 35

---------------------------------------------------------- Real data ------------------------------------------------------------------------

Today:
               time  temperature  relative_humidity  apparent_temperature  weather_code               region  consumption
2019-01-01 00:00:00         6.60              88.33                  3.33             2 Auvergne-Rhône-Alpes       8984.0
2019-01-01 00:30:00         6.38              89.17                  3.10             2 Auvergne-Rhône-Alpes       8815.0
2019-01-01 01:00:00         6.17              90.00                  2.87             2 Auvergne-Rhône-Alpes       8576.0
2019-01-01 01:30:00         5.90              91.17                  2.60             2 Auvergne-Rhône-Alpes       8526.0

Tomorrow:
               time  temperature  relative_humidity  apparent_temperature  weather_code               region  consumption
2019-01-02 00:00:00         4.70              83.33                  0.70         

<br>

In [6]:
mapes = []
real_consumption = Auvergne_data[Auvergne_data['time'].dt.date == target_date + datetime.timedelta(days=1)]['consumption'].values
for i, escenario in enumerate(scenarios):
    scenario_consumption = escenario['consumption'].values
    mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
    mapes.append(mape)

average_mape = np.mean(mapes)
print(f"Average MAPE: {average_mape:.2f}%")

Average MAPE: 8.22%


<br>

### Testing all the validation test

In [ ]:
regions = data['region'].unique()

for region in regions:
    mapes = []
    
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios, _ = generate_scenarios_code(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        day_mapes = []
        for i, scenario in enumerate(scenarios):
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    print(f"Average MAPE for {region}: {round(average_mape, 2)}")

<br>

#### Parallelization

Code executed in spyder.

In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
import gower  

data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

def generate_scenarios_code(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)

    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature', 'weather_code']
    
    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]
    
        distances = gower.gower_matrix(scaled_test, scaled_train_subset)
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

def calculate_region_mape(region):
    mapes = []
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios = generate_scenarios_code(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        if len(real_consumption) == 0:
            continue

        day_mapes = []
        for scenario in scenarios:
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    return (region, round(average_mape, 2))

if __name__ == '__main__':
    regions = data['region'].unique()
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(calculate_region_mape, regions)

    for result in results:
        print(f"Average MAPE for {result[0]}: {result[1]}")

**Output**:

- Average MAPE for Auvergne-Rhône-Alpes: 11.2

- Average MAPE for Bourgogne-Franche-Comté: 13.31

- Average MAPE for Bretagne: 13.33

- Average MAPE for Centre-Val de Loire: 12.96

- Average MAPE for Grand-Est: 11.79

- Average MAPE for Hauts-de-France: 10.07

- Average MAPE for Ile-de-France: 11.65

- Average MAPE for Normandie: 10.78

- Average MAPE for Nouvelle-Aquitaine: 10.6

- Average MAPE for Occitanie: 10.39

- Average MAPE for PACA: 7.91

- Average MAPE for Pays-de-la-Loire: 14.81

<br>

**Total average MAPE: 11.57**

<br>
<br>

Currently, the 'weather_code' variable comprises nearly 100 distinct values. Let's see if our accuracy improves by grouping them based on the WMO code table: https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM

In [7]:
def group_weather(code):
    # No precipitation, fog, ice fog, duststorm, sandstorm, drifting or blowing snow 
    if code >= 0 and code <= 19: 
        return 0
    # Precipitation, fog, ice fog or thunderstorm 
    elif code >= 20 and code <= 29:
        return 1
    # Duststorm, sandstorm, drifting or blowing snow
    elif code >= 30 and code <= 39:
        return 2
    # Fog or ice fog at the time of observation
    elif code >= 40 and code <= 49:
        return 3
    # Drizzle
    elif code >= 50 and code <= 59:
        return 4
    # Rain
    elif code >= 60 and code <= 69:
        return 5
    # Solid precipitation not in showers
    elif code >= 70 and code <= 79:
        return 6
    # Showery precipitation or precipitation with current or recent thunderstorm
    elif code >= 80 and code <= 99:
        return 7
    else:
        return 8 

data['weather_code_group'] = data['weather_code'].apply(group_weather)
train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

data.head(5)

,time,temperature,relative_humidity,apparent_temperature,weather_code,region,consumption,weather_code_group
0,2013-01-01 00:00:00,4.83,85.33,1.03,2,Auvergne-Rhône-Alpes,8173.0,0
1,2013-01-01 00:30:00,4.88,85.00,1.07,2,Auvergne-Rhône-Alpes,8173.0,0
2,2013-01-01 01:00:00,4.93,84.67,1.10,2,Auvergne-Rhône-Alpes,7944.0,0
3,2013-01-01 01:30:00,4.93,84.00,1.08,2,Auvergne-Rhône-Alpes,7896.0,0
4,2013-01-01 02:00:00,4.93,83.33,1.07,2,Auvergne-Rhône-Alpes,7882.0,0


<br>

In [8]:
def generate_scenarios_group(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)

    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature', 'weather_code_group']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]
    
        distances = gower.gower_matrix(scaled_test, scaled_train_subset)
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []
    similar_days = []
    
    for idx in real_indices:
        day = train_data.iloc[idx:idx+48] 
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                similar_days.append(day)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios, similar_days

<br>

In [9]:
# Example for Auvergne-Rhône-Alpes (2019-01-01)
target_date = datetime.date(2019, 1, 1)
scenarios, similar_days = generate_scenarios_group(train_data, 'Auvergne-Rhône-Alpes', target_date)

print('Number of scenarios:', len(scenarios))

print("\n---------------------------------------------------------- Real data ------------------------------------------------------------------------")
Auvergne_data = train_data[train_data['region'] == 'Auvergne-Rhône-Alpes']
print('\nToday:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date].head(4).to_string(index=False))
print('\nTomorrow:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date + datetime.timedelta(days=1)].head(4).to_string(index=False))

for i, escenario in enumerate(scenarios[:3]):
    print(f"\n---------------------------------------------------------- Scenario {i+1} ------------------------------------------------------------------------")
    print('\nSimilar day:')
    print(similar_days[i].head(4).to_string(index=False)) 
    print('\nNext day:')
    print(escenario.head(4).to_string(index=False)) 

Number of scenarios: 35

---------------------------------------------------------- Real data ------------------------------------------------------------------------

Today:
               time  temperature  relative_humidity  apparent_temperature  weather_code               region  consumption  weather_code_group
2019-01-01 00:00:00         6.60              88.33                  3.33             2 Auvergne-Rhône-Alpes       8984.0                   0
2019-01-01 00:30:00         6.38              89.17                  3.10             2 Auvergne-Rhône-Alpes       8815.0                   0
2019-01-01 01:00:00         6.17              90.00                  2.87             2 Auvergne-Rhône-Alpes       8576.0                   0
2019-01-01 01:30:00         5.90              91.17                  2.60             2 Auvergne-Rhône-Alpes       8526.0                   0

Tomorrow:
               time  temperature  relative_humidity  apparent_temperature  weather_code               re

<br>

In [10]:
mapes = []
real_consumption = Auvergne_data[Auvergne_data['time'].dt.date == target_date + datetime.timedelta(days=1)]['consumption'].values

for i, escenario in enumerate(scenarios):
    scenario_consumption = escenario['consumption'].values
    mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
    mapes.append(mape)

average_mape = np.mean(mapes)
print(f"Average MAPE: {average_mape:.2f}%")

Average MAPE: 5.96%


<br>

### Testing all the validation test


In [ ]:
regions = data['region'].unique()

for region in regions:
    mapes = []
    
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios, _ = generate_scenarios_group(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        day_mapes = []
        for i, scenario in enumerate(scenarios):
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    print(f"Average MAPE for {region}: {round(average_mape, 2)}")

<br>

#### Parallelization

Code executed in spyder.

In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
import gower  

data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

def group_weather(code):
    if code >= 0 and code <= 19: 
        return 0
    elif code >= 20 and code <= 29:
        return 1
    elif code >= 30 and code <= 39:
        return 2
    elif code >= 40 and code <= 49:
        return 3
    elif code >= 50 and code <= 59:
        return 4
    elif code >= 60 and code <= 69:
        return 5
    elif code >= 70 and code <= 79:
        return 6
    elif code >= 80 and code <= 99:
        return 7
    else:
        return 8 

data['weather_code_group'] = data['weather_code'].apply(group_weather)

train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

def generate_scenarios_group(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)

    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature', 'weather_code_group']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]

        distances = gower.gower_matrix(scaled_test, scaled_train_subset)
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

def calculate_region_mape(region):
    mapes = []
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios = generate_scenarios_group(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        if len(real_consumption) == 0:
            continue
                    
        day_mapes = []
        for scenario in scenarios:
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    return (region, round(average_mape, 2))

if __name__ == '__main__':
    regions = data['region'].unique()
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(calculate_region_mape, regions)

    for result in results:
        print(f"Average MAPE for {result[0]}: {result[1]}")

**Output**:

- Average MAPE for Auvergne-Rhône-Alpes: 10.91

- Average MAPE for Bourgogne-Franche-Comté: 13.33

- Average MAPE for Bretagne: 13.4

- Average MAPE for Centre-Val de Loire: 12.79

- Average MAPE for Grand-Est: 11.84

- Average MAPE for Hauts-de-France: 10.13

- Average MAPE for Ile-de-France: 11.67

- Average MAPE for Normandie: 10.84

- Average MAPE for Nouvelle-Aquitaine: 10.58

- Average MAPE for Occitanie: 10.16

- Average MAPE for PACA: 7.72

- Average MAPE for Pays-de-la-Loire: 14.46

<br>

**Total average MAPE: 11.48**

<br>

### Not using weather_code

In [11]:
def generate_scenarios_no_code(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)

    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]
    
        distances = gower.gower_matrix(scaled_test, scaled_train_subset)
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_test))]

        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
        
    n_scenarios = n*(2*k+1)
    scenarios = []
    similar_days = []
    
    for idx in real_indices:
        day = train_data.iloc[idx:idx+48] 
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                similar_days.append(day)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios, similar_days

<br>

In [12]:
# Example for Auvergne-Rhône-Alpes (2019-01-01)
target_date = datetime.date(2019, 1, 1)
scenarios, similar_days = generate_scenarios_no_code(train_data, 'Auvergne-Rhône-Alpes', target_date)

print('Number of scenarios:', len(scenarios))

print("\n---------------------------------------------------------- Real data ------------------------------------------------------------------------")
Auvergne_data = train_data[train_data['region'] == 'Auvergne-Rhône-Alpes']
print('\nToday:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date - datetime.timedelta(days=1)].head(4).to_string(index=False))
print('\nTomorrow:')
print(Auvergne_data[Auvergne_data['time'].dt.date == target_date].head(4).to_string(index=False))

for i, escenario in enumerate(scenarios[:3]):
    print(f"\n---------------------------------------------------------- Scenario {i+1} ------------------------------------------------------------------------")
    print('\nSimilar day:')
    print(similar_days[i].head(4).to_string(index=False)) 
    print('\nNext day:')
    print(escenario.head(4).to_string(index=False)) 

Number of scenarios: 35

---------------------------------------------------------- Real data ------------------------------------------------------------------------

Today:
               time  temperature  relative_humidity  apparent_temperature  weather_code               region  consumption  weather_code_group
2018-12-31 00:00:00         4.57              86.33                  1.07             2 Auvergne-Rhône-Alpes       9066.0                   0
2018-12-31 00:30:00         4.48              86.33                  1.10             2 Auvergne-Rhône-Alpes       8868.0                   0
2018-12-31 01:00:00         4.40              86.33                  1.13             2 Auvergne-Rhône-Alpes       8641.0                   0
2018-12-31 01:30:00         4.35              86.67                  1.10             2 Auvergne-Rhône-Alpes       8623.0                   0

Tomorrow:
               time  temperature  relative_humidity  apparent_temperature  weather_code               re

<br>

In [13]:
mapes = []
real_consumption = Auvergne_data[Auvergne_data['time'].dt.date == target_date + datetime.timedelta(days=1)]['consumption'].values

for i, escenario in enumerate(scenarios):
    scenario_consumption = escenario['consumption'].values
    mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
    mapes.append(mape)

average_mape = np.mean(mapes)
print(f"Average MAPE: {average_mape:.2f}%")

Average MAPE: 5.55%


<br>

### Testing all the validation test

In [ ]:
regions = data['region'].unique()

for region in regions:
    mapes = []
    
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios, _ = generate_scenarios_no_code(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        day_mapes = []
        for i, scenario in enumerate(scenarios):
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    print(f"Average MAPE for {region}: {round(average_mape, 2)}")

<br>

#### Parallelization

Code executed in spyder.

In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
import gower  

data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

def generate_scenarios_no_code(data, region, date, n=5, k=3):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)
    
    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]

        distances = gower.gower_matrix(scaled_test, scaled_train_subset)
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

def calculate_region_mape(region):
    mapes = []
    start_date = datetime.date(2019, 1, 1)
    end_date = datetime.date(2019, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios = generate_scenarios_no_code(train_data, region, date)
        real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

        if len(real_consumption) == 0:
            continue
                    
        day_mapes = []
        for scenario in scenarios:
            scenario_consumption = scenario['consumption'].values
            mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
            day_mapes.append(mape)

        average_day_mape = np.mean(day_mapes)
        mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    return (region, round(average_mape, 2))

if __name__ == '__main__':
    regions = data['region'].unique()
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(calculate_region_mape, regions)

    for result in results:
        print(f"Average MAPE for {result[0]}: {result[1]}")

**Output**:

- Average MAPE for Auvergne-Rhône-Alpes: 10.92

- Average MAPE for Bourgogne-Franche-Comté: 13.43

- Average MAPE for Bretagne: 13.35

- Average MAPE for Centre-Val de Loire: 13.06

- Average MAPE for Grand-Est: 11.69

- Average MAPE for Hauts-de-France: 10.17

- Average MAPE for Ile-de-France: 11.79

- Average MAPE for Normandie: 10.77

- Average MAPE for Nouvelle-Aquitaine: 10.52

- Average MAPE for Occitanie: 10.29

- Average MAPE for PACA: 7.76

- Average MAPE for Pays-de-la-Loire: 14.37

<br>

**Total average MAPE: 11.51**

<br>

<br>

|              | With 'weather_code' | With 'weather_code_group' | Without 'weather_code' |
|--------------|------------------|------------------------|------------------|
| MAPE         |       11.57          |         11.48               |      11.51            |

<br>

**Best approach**: With 'weather_code_group'.

However, as the results using or not 'weather_code_group' are nearly identical, **we will not include these variables** in our analysis. 

<br>

## Hyperparameter tunning

In [14]:
def generate_scenarios(data, region, date, n, k, dist):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)
    
    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]

        distances = cdist(scaled_test[features], scaled_train_subset[features], metric=dist) 
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

<br>

In [ ]:
param_grid = {
        'n': [3, 5, 7],
        'k': [1, 3, 5],
        'dist': ['euclidean', 'cityblock']
    }

param_combinations = list(ParameterGrid(param_grid))

best_mape = float('inf')
best_params = None

for params in param_combinations:
    mapes = []
    
    for region in regions:
        start_date = datetime.date(2019, 1, 1)
        end_date = datetime.date(2019, 12, 31)
        validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
        
        for date in validation_dates:
            scenarios, _ = generate_scenarios(train_data, region, date, n=params['n'], k=params['k'])
            real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

            day_mapes = []
            for scenario in scenarios:
                scenario_consumption = scenario['consumption'].values
                mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
                day_mapes.append(mape)

            average_day_mape = np.mean(day_mapes)
            mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    print(f"Parameters: {params}, MAPE: {average_mape}")
    
    if average_mape < best_mape:
        best_mape = average_mape
        best_params = params

print("Best parameters:", best_params)
print("Best MAPE:", best_mape)

<br>

#### Parallelization

Code executed in spyder.

In [ ]:
import multiprocessing
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from scipy.spatial.distance import cdist

data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

train_data = data[data['time'].dt.year <= 2019]
test_data = data[data['time'].dt.year == 2020]  

def generate_scenarios(data, region, date, n, k, dist):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)
    
    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature']

    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]

        distances = cdist(scaled_test[features], scaled_train_subset[features], metric=dist) 
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_train_subset))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

def calculate_mape_for_params(params, regions, train_data):
    mapes = []
    for region in regions:
        start_date = datetime.date(2019, 1, 1)
        end_date = datetime.date(2019, 12, 31)
        validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
        
        for date in validation_dates:
            scenarios = generate_scenarios(train_data, region, date, n=params['n'], k=params['k'], dist=params['dist'])
            real_consumption = train_data[(train_data['region'] == region) & (train_data['time'].dt.date == date + datetime.timedelta(days=1))]['consumption'].values

            if len(real_consumption) == 0:
                continue
                    
            day_mapes = []
            for scenario in scenarios:
                scenario_consumption = scenario['consumption'].values
                mape = np.mean(np.abs((real_consumption - scenario_consumption) / real_consumption)) * 100
                day_mapes.append(mape)

            average_day_mape = np.mean(day_mapes)
            mapes.append(average_day_mape)

    average_mape = np.mean(mapes)
    print(f"Parameters: {params}, MAPE: {average_mape}")
    return average_mape

if __name__ == '__main__':
    regions = data['region'].unique()
    
    param_grid = {
        'n': [3, 5, 7],
        'k': [1, 3, 5],
        'dist': ['euclidean', 'cityblock']
    }
    
    param_combinations = list(ParameterGrid(param_grid))
    
    best_mape = float('inf')
    best_params = None
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.starmap(calculate_mape_for_params, [(params, regions, train_data) for params in param_combinations])
    
    for i, result in enumerate(results):
        if result < best_mape:
            best_mape = result
            best_params = param_combinations[i]
    
    print("Best parameters:", best_params)
    print("Best MAPE:", best_mape)

**Output**:

- Parameters: {'dist': 'euclidean', 'k': 1, 'n': 3}, MAPE: 11.09

- Parameters: {'dist': 'euclidean', 'k': 1, 'n': 5}, MAPE: 11.11

- Parameters: {'dist': 'euclidean', 'k': 1, 'n': 7}, MAPE: 11.12

- Parameters: {'dist': 'euclidean', 'k': 3, 'n': 3}, MAPE: 11.63

- Parameters: {'dist': 'euclidean', 'k': 3, 'n': 5}, MAPE: 11.65

- Parameters: {'dist': 'euclidean', 'k': 3, 'n': 7}, MAPE: 11.66

- Parameters: {'dist': 'euclidean', 'k': 5, 'n': 3}, MAPE: 12.25

- Parameters: {'dist': 'euclidean', 'k': 5, 'n': 5}, MAPE: 12.26

- Parameters: {'dist': 'euclidean', 'k': 5, 'n': 7}, MAPE: 12.28

- Parameters: {'dist': 'cityblock', 'k': 1, 'n': 3}, MAPE: **10.99**

- Parameters: {'dist': 'cityblock', 'k': 1, 'n': 5}, MAPE: 11.04

- Parameters: {'dist': 'cityblock', 'k': 1, 'n': 7}, MAPE: 11.05

- Parameters: {'dist': 'cityblock', 'k': 3, 'n': 3}, MAPE: 11.54

- Parameters: {'dist': 'cityblock', 'k': 3, 'n': 5}, MAPE: 11.59

- Parameters: {'dist': 'cityblock', 'k': 3, 'n': 7}, MAPE: 11.60

- Parameters: {'dist': 'cityblock', 'k': 5, 'n': 3}, MAPE: 12.16

- Parameters: {'dist': 'cityblock', 'k': 5, 'n': 5}, MAPE: 12.21

- Parameters: {'dist': 'cityblock', 'k': 5, 'n': 7}, MAPE: 12.22


<br>

**Best parameters: {'dist': 'cityblock', 'k': 1, 'n': 3}**

**Best MAPE: 10.99**

<br>

## Generation of scenarios with best parameters

In [ ]:
regions = data['region'].unique()

scenarios_final = {}
for region in regions:  
    start_date = datetime.date(2020, 1, 1)
    end_date = datetime.date(2020, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios = generate_scenarios(train_data, region, date, 3, 1, 'cityblock')
        scenarios_final[(region, date)] = scenarios

with open('scenarios_final.pkl', 'wb') as scenarios_file:
    pickle.dump(scenarios_final, scenarios_file)

<br>

#### Parallelization

Code executed in spyder.

In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
import multiprocessing
import pickle

data = pd.read_csv("final_data.csv")
data['time'] = pd.to_datetime(data['time'])

def generate_scenarios(data, region, date, n, k, dist):
    region_data = data[data['region'] == region]
    previous_day = date - datetime.timedelta(days=1)
    
    train_data = region_data[region_data['time'].dt.date < previous_day]
    test = region_data[region_data['time'].dt.date == previous_day]

    features = ['temperature', 'relative_humidity', 'apparent_temperature']
    
    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data[features])
    scaled_test = scaler.transform(test[features])
    
    scaled_train_data = pd.DataFrame(scaled_train_data, columns=features)
    scaled_train_data[features] = scaled_train_data[features].apply(pd.to_numeric, errors='coerce')
    
    scaled_test = pd.DataFrame(scaled_test, columns=features)
    scaled_test[features] = scaled_test[features].apply(pd.to_numeric, errors='coerce')
    
    day_distances = []
    for i in range(int(len(train_data)/48)):
        scaled_train_subset = scaled_train_data.iloc[i*48:(i+1)*48]
        distances = cdist(scaled_test, scaled_train_subset, metric=dist) 
        element_distances = distances[np.arange(len(scaled_test)), np.arange(len(scaled_test))]
        sum_distance = np.sum(element_distances)
        day_distances.append(sum_distance)

    day_distances = np.array(day_distances)
   
    similar_indices = np.argsort(day_distances)[:n*2] 
    real_indices = [idx*48 for idx in similar_indices]
    
    n_scenarios = n*(2*k+1)
    scenarios = []

    for idx in real_indices:
        for i in range(48-k, 48+k+1):
            variation = train_data.iloc[idx+i:idx+i+48] 
            if len(variation) == 48:  
                scenarios.append(variation)
                   
            if len(scenarios) >= n_scenarios:
                break
            
        if len(scenarios) >= n_scenarios:
            break
            
    return scenarios

def calculate_region_scenarios(region):
    scenarios_final = {}
    start_date = datetime.date(2020, 1, 1)
    end_date = datetime.date(2020, 12, 31)
    validation_dates = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    
    for date in validation_dates:
        scenarios = generate_scenarios(data, region, date, 3, 1, 'cityblock')
        scenarios_final[(region, date)] = scenarios

    return scenarios_final

if __name__ == '__main__':
    regions = data['region'].unique()
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(calculate_region_scenarios, regions)

    all_scenarios = {}  
    
    for region_scenarios in results:
        all_scenarios.update(region_scenarios)

    with open('scenarios_final.pkl', 'wb') as scenarios_file:
        pickle.dump(all_scenarios, scenarios_file)    